In [10]:
%%time
import requests
import pandas as pd
import logging
import pandas_datareader.data as web
import datetime
import sys
import os
from dotenv import load_dotenv
from pathlib import Path

dls = "https://cloud-index-data.bvp.com/BVP-Nasdaq-Emerging-Cloud-Index.xlsx"
resp = requests.get(dls)
output = open('BVP-Nasdaq-Emerging-Cloud-Index.xlsx', 'wb')
output.write(resp.content)
output.close()

fndmntls_df = pd.read_excel('BVP-Nasdaq-Emerging-Cloud-Index.xlsx', sheet_name='Index Constituents')
fndmntls_df.columns = fndmntls_df.iloc[5]
fndmntls_df = fndmntls_df.iloc[8:, 1:-1]
fndmntls_df.dropna(subset=['Symbol'], inplace=True)
current_tickers = ','.join(fndmntls_df['Symbol'])

load_dotenv(verbose=True)
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)
# key = os.getenv("SANDBOX_KEY")
# batch_url = 'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={}&types=chart&chartCloseOnly=True&token={}&range=5y'.format(current_tickers, key)
key = os.getenv("IEX_KEY")
batch_url = 'https://cloud.iexapis.com/stable/stock/market/batch?symbols={}&types=chart&chartCloseOnly=True&token={}&range=5y&includeToday=True'.format(current_tickers, key)
resp = requests.get(batch_url)
d = pd.DataFrame(resp.json())
os.remove('BVP-Nasdaq-Emerging-Cloud-Index.xlsx')

Wall time: 17.9 s


In [11]:
d = d.T
d.head()

,chart
TWOU,"[{'date': '2016-01-06', 'close': 26.38, 'volum..."
ADBE,"[{'date': '2016-01-06', 'close': 91.02, 'volum..."
PLAN,"[{'date': '2018-10-12', 'close': 24.3, 'volume..."
APPF,"[{'date': '2016-01-06', 'close': 13.94, 'volum..."
TEAM,"[{'date': '2016-01-06', 'close': 27.05, 'volum..."


In [12]:
d.iloc[0,0][:2]

[{'date': '2016-01-06',
  'close': 26.38,
  'volume': 297213,
  'changeOverTime': 0,
  'marketChangeOverTime': 0,
  'label': 'Jan 6, 16',
  'change': 0,
  'changePercent': 0},
 {'date': '2016-01-07',
  'close': 25.23,
  'volume': 635223,
  'changeOverTime': -0.04359363153904468,
  'marketChangeOverTime': -0.04359363153904468,
  'label': 'Jan 7, 16',
  'change': -1.1499999999999986,
  'changePercent': -0.0436}]

In [13]:
%%time
price_df = pd.DataFrame(columns=['ticker','date','close','volume'])
n=0
ticker_count=0
for idx, row in d.iterrows():
    ticker = idx
    print(ticker_count)
    data = row['chart']
    for day in data:
        price_df.loc[n]=[ticker,day['date'],day['close'],day['volume']]
        n+=1
    ticker_count+=1
price_df.head()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
Wall time: 5min 37s


,ticker,date,close,volume
0,TWOU,2016-01-06,26.38,297213
1,TWOU,2016-01-07,25.23,635223
2,TWOU,2016-01-08,24.32,364477
3,TWOU,2016-01-11,23.72,405094
4,TWOU,2016-01-12,23.27,1012093


In [14]:
price_df.to_csv('Historical_Prices_updated.csv',index=False)